In [1]:
import pandas as pd
import sys
sys.path.append(r"C:\_Github\structural_engineering_toolbox")
from modules import etabs_api, as3600_wall_design

etabs = etabs_api.etabs_api()

In [2]:
load_cases = ['G', 'Q', 'RSS', 'WLX', 'WLY']
story_names = etabs.get_story_names()
start_phz = 'GR'
above_phz = 2
phz_stories = as3600_wall_design.get_phz_stories(start_phz, above_phz, story_names)
vcts = [300, 200]
hcts = [300, 200]
piers = etabs.get_piers(load_cases)

In [3]:
designed_piers = as3600_wall_design.full_wall_design(
    walls=piers,
    phz_levels=phz_stories,
    load_cases=load_cases,
    vcts=vcts,
    hcts=hcts,
    story_names=story_names,
    wall_type='In-situ',
    mu_sp=2.6
)

In [43]:
def piers_as_walls_dataframe(walls:list[dict]) -> pd.DataFrame:
    selected_keys = [
            'pier_name',
            'story_name',
            'story_height',
            'thickness_bot',
            'width_bot',
            'fc',
            'G+0.3Q (MPa)',
            'G+0.3Q+RS (C)(MPa)',
            'G+0.3Q-RS (T)(MPa)',
            'Axial Load Ratio',
            'Slenderness Ratio',
            'Rho crit.',
            'Rho typ.',
            'db Vert',
            's Vert',
            "0.15f'c",
            "0.2f'c",
            "0.585f'c",
            'BE Width',
            'Lig Dia',
            'Lig Cts',
            'EQ Shear',
            'Vuc',
            'Vus',
            'phiVu',
            'db Horiz',
            's Horiz'
            ]
    
    df = pd.DataFrame([{key: wall[key] for key in selected_keys} for wall in walls])

    return df

df = piers_as_walls_dataframe(designed_piers)

In [62]:
color_mapping = {
    'Axial Load Ratio': {"<0.2, OK": 'green', "Increase f'c or wall thickness": 'red'},
    'Slenderness Ratio': {"<20, OK": 'green', "Slender wall. Increase thickness": 'red'}
}

def highlight_cells(val, col, color_mapping):
    return f'background-color: {color_mapping.get(col, {}).get(val, "white")}'

def apply_style(df, color_mapping):
    styled_df = df.style
    for col in ['Axial Load Ratio', 'Slenderness Ratio']:
        styled_df = styled_df.applymap(lambda x, col=col: highlight_cells(x, col, color_mapping), subset=[col])
    return styled_df

styled_df = apply_style(df, color_mapping)
styled_df

TypeError: apply_style() missing 1 required positional argument: 'color_mapping'